Lending Club Project I: Exploratory Data Analysis

We downloaded data at https://www.lendingclub.com/info/download-data.action
Then chose Year **2018 Q1**. Put the downloaded csv file in the same directory as this jupyter notebook.


In [1]:
# Some important packages to import

import pandas as pd
pd.set_option('display.max_columns', None) # you can see all the columns of a dataframe table

import numpy as np
import math
import time
from scipy import stats
from collections import Counter

from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use("ggplot") # ggplot in R

import warnings
warnings.filterwarnings('ignore')

# Warming up

In [2]:
# Read data
loan = pd.read_csv('E:/GitHub/DS701-1904/data/LoanStats_2018Q1.csv', header=1)

In [3]:
loan.head(5)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,10000.0,10000.0,10000.0,60 months,14.07%,233.05,C,C3,Teacher,4 years,RENT,42000.0,Source Verified,Mar-2018,Current,n,NaN,NaN,major_purchase,Major purchase,341xx,FL,24.69,0.0,Oct-2004,0.0,32.0,NaN,17.0,0.0,707.0,15.7%,34.0,w,9035.79,9035.79,1897.19,1897.19,964.21,932.98,0.0,0.0,0.0,Dec-2018,233.05,Jan-2019,Dec-2018,0.0,40.0,1.0,Individual,NaN,NaN,NaN,0.0,0.0,93913.0,0.0,15.0,0.0,0.0,54.0,93206.0,116.0,0.0,1.0,707.0,111.0,4500.0,0.0,0.0,0.0,1.0,5524.0,3793.0,15.7,0.0,0.0,161.0,88.0,18.0,18.0,0.0,18.0,32.0,18.0,32.0,14.0,1.0,1.0,2.0,4.0,30.0,2.0,4.0,1.0,17.0,0.0,0.0,0.0,0.0,43.8,0.0,0.0,0.0,84930.0,93913.0,4500.0,80430.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,2600.0,2600.0,2600.0,36 months,7.96%,81.43,A,A5,Occupational Therapist,3 years,MORTGAGE,62000.0,Not Verified,Mar-2018,Current,n,NaN,NaN,other,Other,481xx,MI,19.61,0.0,Aug-2011,0.0,NaN,NaN,8.0,0.0,11776.0,74.5%,9.0,w,2074.45,2074.45,649.14,649.14,525.55,123.59,0.0,0.0,0.0,Dec-2018,81.43,Jan-2019,Dec-2018,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,241067.0,0.0,2.0,0.0,0.0,34.0,56811.0,13.0,0.0,1.0,7283.0,56.0,15800.0,0.0,0.0,0.0,1.0,30133.0,588.0,94.4,0.0,0.0,79.0,64.0,19.0,19.0,1.0,45.0,NaN,NaN,NaN,0.0,2.0,4.0,2.0,2.0,3.0,5.0,5.0,4.0,8.0,0.0,0.0,0.0,0.0,100.0,100.0,0.0,0.0,259780.0,68587.0,10500.0,62185.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,10000.0,10000.0,10000.0,36 months,6.07%,304.54,A,A2,Credit Director,10+ years,OWN,175000.0,Source Verified,Mar-2018,Current,n,NaN,NaN,house,Home buying,117xx,NY,4

In [4]:
loan.shape

(107866, 145)

In [5]:
# check missing value in row dimension
row_missing = loan.isnull().sum(axis=1)/loan.shape[1]
print (row_missing[row_missing>0.8])

# remove 107864 and 107865
loan.drop(labels=[107864, 107865], axis = 'index', inplace = True)

107864    0.993103
107865    0.993103
dtype: float64


In [6]:
0.993103 * 145

143.999935

In [7]:
# create a dataframe to store number of missing value in column dimension
missing = pd.DataFrame(loan.isnull().sum(axis=0), columns=['missing_num'])
missing.reset_index(level=0, inplace=True)

missing.head(3)

,index,missing_num
0,id,107864
1,member_id,107864
2,loan_amnt,0


In [8]:
LC_dict = pd.read_excel("E:/GitHub/DS701-1904/data/LCDataDictionary.xlsx")
LC_dict.head()

,LoanStatNew,Description
0,acc_now_delinq,The number of accounts on which the borrower i...
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan...
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by th...


In [9]:
# Read Lending Club Data Dictionary
LC_dict = LC_dict.rename(columns={'LoanStatNew':'index'})
LC_dict.head()

,index,Description
0,acc_now_delinq,The number of accounts on which the borrower i...
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan...
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by th...


In [10]:
# merge missing df to LC dictionary
missing_explanation = pd.merge(missing, LC_dict, how='left', on='index')
missing_explanation.head(3)

,index,missing_num,Description
0,id,107864,A unique LC assigned ID for the loan listing.
1,member_id,107864,A unique LC assigned Id for the borrower member.
2,loan_amnt,0,The listed amount of the loan applied for by t...


# Missing 80% or more

In [11]:
# 80% or more missing value for feature dimension
num_80 = 0.8 * loan.shape[0]
missing_80_plus = missing_explanation[
    missing_explanation['missing_num'] >= num_80].sort_values(
    by=['missing_num'], ascending=False)

missing_80_plus.head(8)

,index,missing_num,Description
0,id,107864,A unique LC assigned ID for the loan listing.
18,url,107864,URL for the LC page with listing data.
19,desc,107864,Loan description provided by the borrower
1,member_id,107864,A unique LC assigned Id for the borrower member.
134,orig_projected_additional_accrued_interest,107771,The original projected additional interest amo...
129,hardship_end_date,107761,The end date of the hardship plan period
125,hardship_status,107761,"Describes if the hardship plan is active, pend..."
126,deferral_term,107761,Amount of months that the borrower is expected...


In [12]:
# take a look at the feature names
sorted(missing_80_plus['index'].tolist())

['annual_inc_joint',
 'debt_settlement_flag_date',
 'deferral_term',
 'desc',
 'dti_joint',
 'hardship_amount',
 'hardship_dpd',
 'hardship_end_date',
 'hardship_last_payment_amount',
 'hardship_length',
 'hardship_loan_status',
 'hardship_payoff_balance_amount',
 'hardship_reason',
 'hardship_start_date',
 'hardship_status',
 'hardship_type',
 'id',
 'member_id',
 'mths_since_last_record',
 'mths_since_recent_bc_dlq',
 'orig_projected_additional_accrued_interest',
 'payment_plan_start_date',
 'revol_bal_joint',
 'sec_app_chargeoff_within_12_mths',
 'sec_app_collections_12_mths_ex_med',
 'sec_app_earliest_cr_line',
 'sec_app_inq_last_6mths',
 'sec_app_mort_acc',
 'sec_app_mths_since_last_major_derog',
 'sec_app_num_rev_accts',
 'sec_app_open_acc',
 'sec_app_open_act_il',
 'sec_app_revol_util',
 'settlement_amount',
 'settlement_date',
 'settlement_percentage',
 'settlement_status',
 'settlement_term',
 'url',
 'verification_status_joint']

## Hardship program

* Hardship plan program specifically targets borrowers who are more likely to return to repaying their loan. 
* Under the plan, borrowers are allowed to temporarily make interest-only payments for a period of 3 months to accommodate an unexpected life event. 
* After 3 months, regular payment terms and obligations resume. 
* Only borrowers who fulfill specific characteristics (such as a demonstrated history of repayment) and who claim a hardship will be offered plans. 
* Importantly, borrowers’ loans must be either current or between 1 and 30 days past due to qualify for a hardship plan.

https://debanked.com/2017/04/lending-club-to-beta-hardship-plans-for-borrowers-and-protect-returns-for-investors/

The attributes are only available for those who applied the program. We can see that more than 90% of the users didn't apply for hardship program. One efficient way to deal with these features is to create a new feature, indicating whether this user applied program or not. Then remove all related attributed.

In [13]:
hardship_df = missing_80_plus[missing_80_plus['index'].str.contains('hardship')]
hardship_df.head(10)

,index,missing_num,Description
129,hardship_end_date,107761,The end date of the hardship plan period
125,hardship_status,107761,"Describes if the hardship plan is active, pend..."
127,hardship_amount,107761,The interest payment that the borrower has com...
128,hardship_start_date,107761,The start date of the hardship plan period
132,hardship_dpd,107761,Account days past due as of the hardship plan ...
131,hardship_length,107761,The number of months the borrower will make sm...
133,hardship_loan_status,107761,Loan Status as of the hardship plan start date
135,hardship_payoff_balance_amount,107761,The payoff balance amount as of the hardship p...
136,hardship_last_payment_amount,107761,The last payment amount as of the hardship pla...
124,hardship_reason,107761,Describes the reason the hardship plan was off...


In [14]:
# create a new feature "is_hardship"
loan['is_hardship'] = loan['hardship_status'].isna()
loan.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,is_hardship
0,NaN,NaN,10000.0,10000.0,10000.0,60 months,14.07%,233.05,C,C3,Teacher,4 years,RENT,42000.0,Source Verified,Mar-2018,Current,n,NaN,NaN,major_purchase,Major purchase,341xx,FL,24.69,0.0,Oct-2004,0.0,32.0,NaN,17.0,0.0,707.0,15.7%,34.0,w,9035.79,9035.79,1897.19,1897.19,964.21,932.98,0.0,0.0,0.0,Dec-2018,233.05,Jan-2019,Dec-2018,0.0,40.0,1.0,Individual,NaN,NaN,NaN,0.0,0.0,93913.0,0.0,15.0,0.0,0.0,54.0,93206.0,116.0,0.0,1.0,707.0,111.0,4500.0,0.0,0.0,0.0,1.0,5524.0,3793.0,15.7,0.0,0.0,161.0,88.0,18.0,18.0,0.0,18.0,32.0,18.0,32.0,14.0,1.0,1.0,2.0,4.0,30.0,2.0,4.0,1.0,17.0,0.0,0.0,0.0,0.0,43.8,0.0,0.0,0.0,84930.0,93913.0,4500.0,80430.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,True
1,NaN,NaN,2600.0,2600.0,2600.0,36 months,7.96%,81.43,A,A5,Occupational Therapist,3 years,MORTGAGE,62000.0,Not Verified,Mar-2018,Current,n,NaN,NaN,other,Other,481xx,MI,19.61,0.0,Aug-2011,0.0,NaN,NaN,8.0,0.0,11776.0,74.5%,9.0,w,2074.45,2074.45,649.14,649.14,525.55,123.59,0.0,0.0,0.0,Dec-2018,81.43,Jan-2019,Dec-2018,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,241067.0,0.0,2.0,0.0,0.0,34.0,56811.0,13.0,0.0,1.0,7283.0,56.0,15800.0,0.0,0.0,0.0,1.0,30133.0,588.0,94.4,0.0,0.0,79.0,64.0,19.0,19.0,1.0,45.0,NaN,NaN,NaN,0.0,2.0,4.0,2.0,2.0,3.0,5.0,5.0,4.0,8.0,0.0,0.0,0.0,0.0,100.0,100.0,0.0,0.0,259780.0,68587.0,10500.0,62185.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,True
2,NaN,NaN,10000.0,10000.0,10000.0,36 months,6.07%,304.54,A,A2,Credit Director,10+ years,OWN,175000.0,Source Verified,Mar-2018,Current,n,NaN,NaN,house,

In [15]:
# remove all hardship-related features
loan = loan.drop(hardship_df['index'].tolist(), axis = 1)

In [16]:
loan.shape

(107864, 135)

## Joint Application Loan Program

* Allows two borrowers to apply together
* Implement a more integrated evaluation of both borrower applicants’ credit profiles
* Additional credit criteria are being added for the secondary applicant
* Borrowers who apply together are able to utilize a higher income
* Investors can benefit given that two borrowers are jointly responsible to pay off the loan
* performed better and demonstrated lower risk on average

https://blog.lendingclub.com/refining-enhancing-joint-application-loan-program/

### Joint features
For 
*  **annual_inc_joint**
*  **verification_status_joint**
*  **dti_joint**
*  **revol_bal_joint**

NA suggest that the application is only made by one borrower. Thus, one simple yet very reasonable way to impute these features is to replace NA by their corresponding non-joint features, e.g. 'annual_inc','verification_status',etc.

In [17]:
joint_features = ['annual_inc','verification_status','dti','revol_bal']

for f in joint_features:
    # fill na by another column
    loan[f + '_new'] = loan[f + '_joint'].fillna(loan[f], inplace=False)
    # check missing value after imputation
    print (f + '_new', ':', loan[f + '_new'].isnull().sum())

annual_inc_new : 0
verification_status_new : 0
dti_new : 0
revol_bal_new : 0


In [18]:
loan.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,deferral_term,payment_plan_start_date,orig_projected_additional_accrued_interest,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,is_hardship,annual_inc_new,verification_status_new,dti_new,revol_bal_new
0,NaN,NaN,10000.0,10000.0,10000.0,60 months,14.07%,233.05,C,C3,Teacher,4 years,RENT,42000.0,Source Verified,Mar-2018,Current,n,NaN,NaN,major_purchase,Major purchase,341xx,FL,24.69,0.0,Oct-2004,0.0,32.0,NaN,17.0,0.0,707.0,15.7%,34.0,w,9035.79,9035.79,1897.19,1897.19,964.21,932.98,0.0,0.0,0.0,Dec-2018,233.05,Jan-2019,Dec-2018,0.0,40.0,1.0,Individual,NaN,NaN,NaN,0.0,0.0,93913.0,0.0,15.0,0.0,0.0,54.0,93206.0,116.0,0.0,1.0,707.0,111.0,4500.0,0.0,0.0,0.0,1.0,5524.0,3793.0,15.7,0.0,0.0,161.0,88.0,18.0,18.0,0.0,18.0,32.0,18.0,32.0,14.0,1.0,1.0,2.0,4.0,30.0,2.0,4.0,1.0,17.0,0.0,0.0,0.0,0.0,43.8,0.0,0.0,0.0,84930.0,93913.0,4500.0,80430.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,True,42000.0,Source Verified,24.69,707.0
1,NaN,NaN,2600.0,2600.0,2600.0,36 months,7.96%,81.43,A,A5,Occupational Therapist,3 years,MORTGAGE,62000.0,Not Verified,Mar-2018,Current,n,NaN,NaN,other,Other,481xx,MI,19.61,0.0,Aug-2011,0.0,NaN,NaN,8.0,0.0,11776.0,74.5%,9.0,w,2074.45,2074.45,649.14,649.14,525.55,123.59,0.0,0.0,0.0,Dec-2018,81.43,Jan-2019,Dec-2018,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,241067.0,0.0,2.0,0.0,0.0,34.0,56811.0,13.0,0.0,1.0,7283.0,56.0,15800.0,0.0,0.0,0.0,1.0,30133.0,588.0,94.4,0.0,0.0,79.0,64.0,19.0,19.0,1.0,45.0,NaN,NaN,NaN,0.0,2.0,4.0,2.0,2.0,3.0,5.0,5.0,4.0,8.0,0.0,0.0,0.0,0.0,100.0,100.0,0.0,0.0,259780.0,68587.0,10500.0,62185.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,True,62000.0,Not Verified,19.61,11776.0
2,NaN,NaN,10000.0,10000.0,10000.0,36 months,6.07%,304.54,A,A2,Credit Director,10+ years,OWN,175000.0,Source Verified,Mar-2018,Current,n,NaN,NaN,house,Home buying,117xx,NY,4.94,0.0,Mar-1994,0.0,NaN,NaN,12.0,0.0,6885.0,11.3%,25.0,w,7932.01,7932.01,2429.58,2429.58,2067.99,361.59,0.0,0.0,0.0,Dec-2018,304.54,Jan-2019,De

In [19]:
# remove joint and non-joint features
loan = loan.drop(joint_features + 
                 [x + '_joint' for x in joint_features], 
                 axis = 1)

### Secondary applicant features

For
* **sec_app_inq_last_6mths**: Credit inquiries in the last 6 months at time of application for the secondary applicant. 
* **sec_app_mort_acc**: Number of mortgage accounts at time of application for the secondary applicant. 
* **sec_app_open_acc**: Number of open trades at time of application for the secondary applicant. (91533 NA values)
* **sec_app_revol_util**: Ratio of total current balance to high credit/credit limit for all revolving accounts. 
* **sec_app_open_act_il**: Number of currently active installment trades at time of application for the secondary applicant. 
* **sec_app_num_rev_accts**: Number of revolving accounts at time of application for the secondary applicant. 
* **sec_app_chargeoff_within_12_mths**: Number of charge-offs within last 12 months at time of application for the secondary applicant. 
* **sec_app_collections_12_mths_ex_med**: Number of collections within last 12 months excluding medical collections at time of application for the secondary applicant. 
* **sec_app_mths_since_last_major_derog**: Months since most recent 90-day or worse rating at time of application for the secondary applicant.
* **sec_app_earliest_cr_line**: Earliest credit line at time of application for the secondary applicant

Most of the "sec_app" features are numerical except the last one. For NAs, the feature is useless. For non-NAs, the feature somewhat reflects the credit information, and should be included in our final model. To contain this information, we can take the average of first and second applicant for each numerical feature. For categorical feature **sec_app_earliest_cr_line**, take the earlier year of first and second applicant. 

'mths_since_last_major_derog' has more than 80000 missing values, so we just remove it, along with the 'sec_app_mths_since_last_major_derog'.

'revol_util' has only 149 missing values, thus we can use median to impute. Note that the value of this feature is like '15.7%', so we have to convert it to number first. 

In [20]:
# drop 'mths_since_last_major_derog' related features
loan.drop(['mths_since_last_major_derog', 
           'sec_app_mths_since_last_major_derog'], 
          axis = 1, 
          inplace=True)

In [21]:
# impute 'revol_util'

# apply function to convert non-nan value into numerical
loan['revol_util'] = loan['revol_util'].apply(
    lambda x: float(x[:-1]) / 100 if str(x)!='nan' else x) #不要最后一位“百分号”再除以100

# impute NA with median value
loan['revol_util'].fillna(
    np.nanmedian(loan['revol_util']),
    inplace = True
)

# remove 'revol_util'
loan.drop(['revol_util','sec_app_revol_util'], axis=1, inplace=True)

In [22]:
first_app_numerical_features = [
     'chargeoff_within_12_mths',
     'collections_12_mths_ex_med',
     'inq_last_6mths',
     'mort_acc',
     'num_rev_accts',
     'open_acc',
     'open_act_il']

first_app_categorical_features = 'earliest_cr_line'

# handle numerical
for f in first_app_numerical_features:
    loan['sec_app_' + f].fillna(loan[f], inplace=True)
    loan[f + '_new'] = (loan['sec_app_' + f] + loan[f]) / 2

# handle categorical variable
loan['sec_app_earliest_cr_line'].fillna(loan['earliest_cr_line'], inplace=True)

loan['earliest_cr_line'] = loan['earliest_cr_line'].apply(lambda x: x.split('-')[1])
loan['sec_app_earliest_cr_line'] = loan['sec_app_earliest_cr_line'].apply(lambda x: x.split('-')[1])

loan['earliest_cr_line_new'] = loan[['earliest_cr_line','sec_app_earliest_cr_line']].min(axis=1)

loan['earliest_cr_line_new'] = loan['earliest_cr_line_new'].apply(lambda x: str(int(x/10)*10))

In [23]:
print(loan['sec_app_earliest_cr_line'])
print(loan['earliest_cr_line_new'])

0         2004
1         2011
2         1994
3         2004
4         2005
5         2000
6         2001
7         2003
8         1989
9         1993
10        2000
11        2004
12        2003
13        1994
14        2005
15        1992
16        2000
17        2006
18        2000
19        1995
20        2001
21        1992
22        2004
23        2003
24        2004
25        2004
26        2005
27        2013
28        1999
29        1991
          ... 
107834    2013
107835    2010
107836    2005
107837    2001
107838    1984
107839    2011
107840    2004
107841    2006
107842    2002
107843    2006
107844    2000
107845    2008
107846    1994
107847    2005
107848    2011
107849    1981
107850    2003
107851    1999
107852    2002
107853    1979
107854    1998
107855    2001
107856    2006
107857    1982
107858    1992
107859    1995
107860    1997
107861    1994
107862    1998
107863    2007
Name: sec_app_earliest_cr_line, Length: 107864, dtype: object
0         2000
1       

In [24]:
# remove all relevant features
for f in first_app_numerical_features + ['earliest_cr_line']:
    loan.drop([f, 'sec_app_'+f], axis=1, inplace=True)

In [25]:
loan.shape

(107864, 119)

## Settlement

For
* **settlement_amount**: The loan amount that the borrower has agreed to settle for
* **settlement_date**: The date that the borrower agrees to the settlement plan
* **settlement_percentage**: The settlement amount as a percentage of the payoff balance amount on the loan
* **settlement_status**: The status of the borrower’s settlement plan. Possible values are: COMPLETE, ACTIVE, BROKEN, CANCELLED, DENIED, DRAFT
* **settlement_term**: The number of months that the borrower will be on the settlement plan
* **debt_settlement_flag_date**: The most recent date that the Debt_Settlement_Flag has been set.

if you're having a problem making your payments, we encourage you to reach out to us before enrolling with a debt settlement company.

Debt settlement companies often claim they can negotiate with your creditors to reduce the amount you owe, but working with a debt settlement company doesn't necessarily mean that your loan will settle. Your loan will continue to accumulate interest, penalties, or fees until it is charged off.

https://help.lendingclub.com/hc/en-us/articles/115011819087-Debt-settlement-companies

Thus, we can replace all NA values with 0 to signify that borrowers are not working with a debt settlement company. For categorical variables, we can remove them, since it contains too much NAs and it makes no sense using them to predict interest rate. 

In [26]:
# fill NAs with 0
loan['settlement_amount'].fillna(0, inplace=True)
loan['settlement_percentage'].fillna(0, inplace=True)
loan['settlement_term'].fillna(0, inplace=True)

# remove other relevant features
loan.drop(['settlement_date','settlement_status','debt_settlement_flag_date'], axis=1, inplace=True)

In [27]:
loan.shape

(107864, 116)

## Other features

In [28]:
other_80_plus = [x for x in missing_80_plus['index'].tolist()
                 if 'hardship' not in x and 
                     'sec_app_' not in x and 
                     '_joint' not in x and 
                     'settlement' not in x]
other_80_plus

['id',
 'url',
 'desc',
 'member_id',
 'orig_projected_additional_accrued_interest',
 'deferral_term',
 'payment_plan_start_date',
 'mths_since_last_record',
 'mths_since_recent_bc_dlq']

In [29]:
# remove them 
loan.drop(other_80_plus, axis=1, inplace=True)

# Missing less than 80%

In [30]:
missing = pd.DataFrame(loan.isnull().sum(axis=0), columns=['missing_num'])
missing.reset_index(level=0, inplace=True)

missing['percentage'] = missing['missing_num'] / loan.shape[0]

In [31]:
missing.head(10)

,index,missing_num,percentage
0,loan_amnt,0,0.000000
1,funded_amnt,0,0.000000
2,funded_amnt_inv,0,0.000000
3,term,0,0.000000
4,int_rate,0,0.000000
5,installment,0,0.000000
6,grade,0,0.000000
7,sub_grade,0,0.000000
8,emp_title,9613,0.089121
9,emp_length,9428,0.087406


In [32]:
# check if all features have missing percentage less than 0.8
missing['percentage'].max()

0.7142141956537863

In [33]:
# find non-zero missing percentage
missing[missing['percentage'] != 0]['index']

8                          emp_title
9                         emp_length
19            mths_since_last_delinq
32                      last_pymnt_d
34                      next_pymnt_d
35                last_credit_pull_d
44                mths_since_rcnt_il
46                           il_util
50                          all_util
56                       avg_cur_bal
57                    bc_open_to_buy
58                           bc_util
60                mo_sin_old_il_acct
64              mths_since_recent_bc
65             mths_since_recent_inq
66    mths_since_recent_revol_delinq
76                  num_tl_120dpd_2m
80                    pct_tl_nvr_dlq
81                  percent_bc_gt_75
Name: index, dtype: object

## Month Type

* **mths_since_last_delinq**: The number of months since the borrower's last delinquency. Missing value occurs because the user has no delinquency record. So we can replace NA by 0.


* **mths_since_rcnt_il**: Months since most recent installment accounts opened. Impute with median value


* **mths_since_recent_bc**: Months since most recent bankcard account opened. Impute with median value


* **mths_since_recent_inq**: Months since most recent inquiry. Impute with median value


* **mths_since_recent_revol_delinq**: Months since most recent revolving delinquency. Remove


* **mo_sin_old_il_acct**: Months since oldest bank installment account opened. Replace NA by 0


* **last_pymnt_d**: Last month payment was received. NA could be another category


* **next_pymnt_d**: Next scheduled payment date. NA could be another category

In [34]:
# replace by 0
loan['mths_since_last_delinq'].fillna(0, inplace=True)
loan['mo_sin_old_il_acct'].fillna(0, inplace=True)

# replace by median
for f in ['mths_since_rcnt_il', 'mths_since_recent_bc', 'mths_since_recent_inq']:
    loan[f].fillna(np.nanmedian(loan[f]), inplace=True)

# remove 
loan.drop('mths_since_recent_revol_delinq', axis=1, inplace=True)

In [35]:
loan['last_pymnt_d'].value_counts()

Dec-2018    86134
Nov-2018     6444
Oct-2018     2320
Aug-2018     2177
Jul-2018     1877
Sep-2018     1832
Jun-2018     1717
May-2018     1537
Apr-2018     1373
Mar-2018     1309
Feb-2018      736
Jan-2018      258
Name: last_pymnt_d, dtype: int64

In [36]:
# replace by other
loan['last_pymnt_d'].fillna('Other', inplace=True)
loan['next_pymnt_d'].fillna('Other', inplace=True)

In [51]:
loan[loan.isnull()]['last_pymnt_amnt'].describe()

count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: last_pymnt_amnt, dtype: float64

In [37]:
# All NA values under last_pymnt_d have last_pymnt_amnt "0"
loan[loan['last_pymnt_d'].isnull()]['last_pymnt_amnt'].describe()

count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: last_pymnt_amnt, dtype: float64

## Others

* 'pct_tl_nvr_dlq', 'all_util', 'avg_cur_bal': Only a small number of missing values, just impute with median value


* 'last_credit_pull_d': The most recent month LC pulled credit for this loan. Impute with majority category


* 'emp_title': The job title supplied by the Borrower when applying for the loan. Originally, it has 37289 levels. Use a basic string preprocessing to lower down levels. For each title, extract the last element splitting by the space and lowercase it. Find top k levels to include as much information as we can. Specify the rest of levels as "Others".


* 'emp_length': Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years. Specify NA as "Other"


* 'il_util': Ratio of total current balance to high credit/credit limit on all installment account. First, create a new feature **il_util_2 = total_bal_il / total_il_high_credit_limit**. Second, remove il_util. Third, impute NA with 0, since NA is caused by 0 denominator.


* 'bc_open_to_buy': Total open to buy on revolving bankcards. Impute with median


* 'bc_util': Ratio of total current balance to high credit/credit limit for all bankcard accounts. Impute with median


* 'num_tl_120dpd_2m': Number of accounts currently 120 days past due (updated in past 2 months). Impute with 0


* 'percent_bc_gt_75': Percentage of all bankcard accounts > 75% of limit. Impute with median



In [52]:
# replace by median
for f in ['pct_tl_nvr_dlq', 'all_util', 'avg_cur_bal', 'bc_open_to_buy','bc_util','percent_bc_gt_75']:
    loan[f].fillna(
        np.nanmedian(loan[f]), 
        inplace=True)

In [54]:
loan['last_credit_pull_d'].value_counts()

Dec-2018    97625
Nov-2018     2251
Oct-2018     1855
Jul-2018     1283
Sep-2018     1218
Aug-2018     1031
Mar-2018      623
Feb-2018      494
May-2018      480
Jan-2018      440
Apr-2018      403
Jun-2018       82
Dec-2017       77
Nov-2017        1
Name: last_credit_pull_d, dtype: int64

In [55]:
# impute by majority
loan['last_credit_pull_d'].fillna(
    loan['last_credit_pull_d'].value_counts().index[0], 
    inplace=True)

In [56]:
# deal with emp_title

# impute with Other
loan['emp_title'].fillna('Other', inplace=True)
# remove space in the beginning and end
loan['emp_title'] = loan['emp_title'].apply(lambda x: x.strip())
# choose the last word as the general title
loan['emp_title'] = loan['emp_title'].apply(lambda x: x.split(' ')[-1].lower() if ' ' in x else x.lower())

# choose top 20 in order to contain half of the variance
top_20_title = loan['emp_title'].value_counts()[:20].index.tolist()
loan['emp_title'] = loan['emp_title'].apply(lambda x: x if x in top_20_title else 'minority')

In [57]:
# deal with emp_length
loan['emp_length'].fillna('Other', inplace=True)

In [58]:
# create a new feature il_util_2
loan['il_util_2'] = loan['total_bal_il'] / loan['total_il_high_credit_limit'] * 100

# loan[['il_util','il_util_2']]

# remove il_util
loan.drop(['il_util'], axis=1, inplace=True)

# impute with 0
loan['il_util_2'].fillna(0, inplace=True)

In [59]:
# deal with num_tl_120dpd_2m
loan['num_tl_120dpd_2m'].fillna(0, inplace=True)

# Missing Value Final Check

In [60]:
loan.isnull().sum().sum()

0

In [61]:
loan.shape

(107864, 106)

In [68]:
#loan.isnull().sum()
loan['loan_amnt'].head()
loan['loan_amnt'].value_counts().sum()
len(loan['loan_amnt'].value_counts())

1482

# Remove 1-value Features

In [47]:
one_value_features = []

for f in loan:
    if len(loan[f].value_counts()) == 1:
        one_value_features.append(f)

loan.drop(one_value_features, axis=1, inplace=True)

# Outliers Detection and Processing

In [48]:
# get numerical features

def get_numerical_features(df):
    
    """
    input: dataframe
    return: numerical feature list
    """
    res = []
    for f in df:
        if np.issubdtype(df[f].dtype, np.number):
#             print (df[f].dtype)
            res.append(f)
    
    return res

numerical_features = get_numerical_features(loan)

In [49]:
# test
len(numerical_features)

79

In [55]:
numerical_features[0]

'loan_amnt'

In [51]:
# detect outlier using IQR

def detect_outlier(data):
    
    """
    input: numerical data, Array
    return: new data without outlier
    """
    tmp = data.copy()
    
    q1, q3 = np.percentile(tmp, [25, 75])
    iqr = q3 - q1
    lower_bound = q1 - (iqr * 1.5)
    upper_bound = q3 + (iqr * 1.5)
    
    has_outlier = (np.max(tmp) > upper_bound) or (np.min(tmp) < lower_bound)
    
    if not has_outlier:
        return tmp
    
    upper_idx = np.where(tmp > upper_bound)
    lower_idx = np.where(tmp < lower_bound)
    tmp[upper_idx] = upper_bound
    tmp[lower_idx] = lower_bound
    
    if np.max(tmp) <= upper_bound and np.min(tmp) >= lower_bound:
        print ("Done with handling outliers")
    else:
        print ("Warning!! Fail handling outliers")
    
    return tmp

In [52]:
for f in numerical_features:
    loan[f] = detect_outlier(loan[f].values)

Done with handling outliers
Done with handling outliers
Done with handling outliers
Done with handling outliers
Done with handling outliers
Done with handling outliers
Done with handling outliers
Done with handling outliers
Done with handling outliers
Done with handling outliers
Done with handling outliers
Done with handling outliers
Done with handling outliers
Done with handling outliers
Done with handling outliers
Done with handling outliers
Done with handling outliers
Done with handling outliers
Done with handling outliers
Done with handling outliers
Done with handling outliers
Done with handling outliers
Done with handling outliers
Done with handling outliers
Done with handling outliers
Done with handling outliers
Done with handling outliers
Done with handling outliers
Done with handling outliers
Done with handling outliers
Done with handling outliers
Done with handling outliers
Done with handling outliers
Done with handling outliers
Done with handling outliers
Done with handling o

# Save Data

In [53]:
loan.to_csv('E:/GitHub/DS701-1904/data/LoanStats_model.csv', encoding='utf-8', index=False)